In [4]:
import numpy as np
import pandas as pd

A = np.array([[-64132, -80820, -84496, -57728, -88595, -83348, -87339], 
    [10149, 12260, 12451, 8634.3, 12705, 11367, 13071], 
    [-537.69, -619.90, -613.61, -442.77, -613.96, -533.98, -648.75], 
    [12.287, 13.27, 12.968, 10.593, 12.782, 11.685, 13.623]])
gmv = np.array([26161096.98, 28859416.46, 27457585.25, 27333511.76, 27931773.23, 19912396.65, 16650378.09])

n = 1000
c_rate = np.tile(np.linspace(0.0, 0.1, n), 7)
week = np.arange(7).repeat(n)
gmv = gmv.repeat(n)
df = pd.DataFrame({'week': week, 'c_rate': c_rate, 'gmv': gmv})
df['roi'] = df.apply(lambda x: A[0][int(x['week'])]*x['c_rate']**3 + A[1][int(x['week'])]*x['c_rate']**2 + A[2][int(x['week'])]*x['c_rate'] + A[3][int(x['week'])], axis=1)
df['dcost'] = df['c_rate']*df['gmv']
df['dgmv'] = df['roi']*df['dcost']
df

,week,c_rate,gmv,roi,dcost,dgmv
0,0,0.0000,26161096.98,12.287000,0.000000e+00,0.000000e+00
1,0,0.0001,26161096.98,12.233279,2.618728e+03,3.203563e+04
2,0,0.0002,26161096.98,12.179761,5.237457e+03,6.379097e+04
3,0,0.0003,26161096.98,12.126445,7.856185e+03,9.526760e+04
4,0,0.0004,26161096.98,12.073332,1.047491e+04,1.264671e+05
...,...,...,...,...,...,...
6995,6,0.0996,16650378.09,-7.620949,1.658371e+06,-1.263836e+07
6996,6,0.0997,16650378.09,-7.685569,1.660038e+06,-1.275833e+07
6997,6,0.0998,16650378.09,-7.750451,1.661704e+06,-1.287896e+07
6998,6,0.0999,16650378.09,-7.815594,1.663371e+06,-1.300023e+07


In [14]:
from ortools.linear_solver import pywraplp

x = {}
solver = pywraplp.Solver.CreateSolver('CBC')
solver.max_time_in_seconds = 240.0
for i in range(len(df)):
    x[i] = solver.NumVar(0.0, 1, 'x[%i]' % (i))

budget = 6000000
solver.Add(solver.Sum([x[i] * df['dcost'].iloc[i] for i in range(0, len(x))]) <= budget)
for a in range(7):
    solver.Add(solver.Sum([x[i] for i in range(a * n, (a + 1) * n)]) == 1)

solver.Minimize(-(solver.Sum([x[i] * df['dgmv'].iloc[i] for i in range(0, len(x))])))
status = solver.Solve()

In [28]:
dcost_total = 0
gmv_total = 0
dgmv_total = 0
week = 0
for i in range(0, len(x)):
    cur_val = x[i].solution_value()
    if cur_val > 0.0:
        week += 1
        c_rate = df['c_rate'][i]
        roi = df['roi'][i]
        print(f'第{week}周的补贴率为{round(100 * c_rate, 2)}%')
        print(f'第{week}周的ROI为{round(100 * roi)}%\n')
        dcost_total += df['dcost'][i]
        gmv_total += df['gmv'][i]
        dgmv_total += df['dgmv'][i]

print(f'总补贴率为{round(100 * dcost_total/gmv_total, 2)}%')
print(f'总ROI为{round(100 * dgmv_total/dcost_total)}%')
print(f'原始GMV为{round(gmv_total)}元')
print(f'总补贴为{round(dcost_total)}元')
print(f'增量GMV为{round(dgmv_total)}元')

第1周的补贴率为1.69%
第1周的ROI为578%

第2周的补贴率为1.59%
第2周的ROI为618%

第3周的补贴率为1.58%
第3周的ROI为604%

第4周的补贴率为5.79%
第4周的ROI为278%

第5周的补贴率为5.68%
第5周的ROI为272%

第6周的补贴率为1.56%
第6周的ROI为596%

第7周的补贴率为5.17%
第7周的ROI为294%

第8周的补贴率为5.98%
第8周的ROI为289%

总补贴率为3.44%
总ROI为363%
原始GMV为201763744元
总补贴为6935854元
增量GMV为25171044元
